In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [33]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [34]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()

In [35]:
df_org=pd.read_csv('train_jqd04QH.csv')
df_test=pd.read_csv('test_KaymcHn.csv')
df_sub=pd.read_csv('sample_submission_sxfcbdx.csv')
df_copy=df_org.copy()

In [36]:
def feature_preprocessing(data):
    x=pd.DataFrame(data.city.value_counts(normalize=True))
    x['city_name']=x.index
    x.reset_index(drop=True,inplace=True)
    x.columns=['count_city','city']
    data=pd.merge(data,x,on='city',how='left')
    
    x=pd.DataFrame(data.education_level.value_counts(normalize=True))
    x['education_level_1']=x.index
    x.columns=['count_eduction_level','education_level']
    x.reset_index(drop=True,inplace=True)
    data=pd.merge(left=data,right=x,on='education_level',how='left')

    x=pd.DataFrame(data.major_discipline.value_counts(normalize=True))
    x['major_discipline1']=x.index
    x.columns=['count_major_discipline_level','major_discipline']
    x.reset_index(drop=True,inplace=True)
    data=pd.merge(left=data,right=x,on='major_discipline',how='left')
    return data


def feature_engineering(data):
    x1=np.nan
    data['nan_main']=data.apply(lambda x:list(x.loc[['education_level','major_discipline','experience','company_size','company_type','last_new_job']].values).count(x1),axis=1)
    data['gender']=data['gender'].fillna(data['gender'].mode()[0])
    data['enrolled_university']=data['enrolled_university'].fillna(data['enrolled_university'].mode()[0])
    data['education_level']=data['education_level'].fillna(data['education_level'].mode()[0])
    data['major_discipline']=data['major_discipline'].fillna(data['major_discipline'].mode()[0])
    data['experience']=data['experience'].fillna(data['experience'].mode()[0])
    data['company_size']=data['company_size'].fillna(data['company_size'].mode()[0])
    data['company_type']=data['company_type'].fillna(data['company_type'].mode()[0])
    data['last_new_job']=data['last_new_job'].fillna(data['last_new_job'].mode()[0])
    return data

In [37]:
df_copy=feature_engineering(df_copy)
df_test=feature_engineering(df_test)
df_copy=feature_preprocessing(df_copy)
df_test=feature_preprocessing(df_test)

In [41]:
cols=df_copy.select_dtypes('object').columns
for i in cols:
    lb=LabelEncoder()
    unique_val=list(set(df_copy[i].unique().tolist()+df_test[i].unique().tolist()))
    lb.fit(unique_val)
    df_copy[i]=lb.transform(df_copy[i])
    df_test[i]=lb.transform(df_test[i])

### Model Selection

In [43]:
from sklearn.ensemble import GradientBoostingClassifier
gbc=GradientBoostingClassifier()

In [44]:
from sklearn.model_selection import cross_val_score
import lightgbm as lgb

X=df_copy.drop(columns=["target","enrollee_id"])
y=df_copy["target"]

results=cross_val_score(gbc,X,y,cv=3,scoring="roc_auc",n_jobs=-1)
results

results.mean()

0.6607310855528953

In [23]:
gbc.fit(X,y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [12]:
df_test=df_test.drop(['enrollee_id'],axis=1)

In [13]:
pred=gbc.predict_proba(df_test)[:,-1]

In [ ]:
df_sub['target']=pred

In [ ]:
df_sub.to_csv('output_gbc_without',index=False)

In [27]:
df_test['count_eduction_level'].unique()

array([0.59777109, 0.02885273, 0.11459046, 0.23779571,        nan,
       0.02099002])